In [41]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

resolution = [480,320] # increase for§ higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [42]:
max_lat = 51.28801
min_lat = 51.281646
max_long = 6.79298
min_long = 6.782016

In [ ]:
import pandas as pd 
df_antennas = pd.read_excel("/Users/andreamaestri/Desktop/tectwin/GEO/indoor-rf-design/src/dusseldorf_antennas.xlsx")
df_antennas = df_antennas[(df_antennas['CellLatitude'] >= min_lat) & (df_antennas['CellLatitude'] <= max_lat) &
                 (df_antennas['CellLongitude'] >= min_long) & (df_antennas['CellLongitude'] <= max_long) & (df_antennas['Technology'] == "T5G")]

# Display the filtered DataFrame
lat_long = df_antennas[["CellLatitude","CellLongitude","ActualAntennaheight"]].values
lat_long

In [ ]:
df_antennas

In [ ]:
import numpy as np 
angles_antennas = df_antennas[["RtsAzimuth","ActualElectricalTilt","ActualMechanicalTilt"]]
angles_antennas = (angles_antennas / 180 * np.pi).values
angles_antennas

In [46]:
from pyproj import Transformer
import numpy as np
transformer = Transformer.from_crs("epsg:4326", 'epsg:32632')

coordinates = np.array(transformer.transform(lat_long[:, 0],lat_long[:, 1]))
delta = (0,0,38)

moved_antennas_x = np.array([ x-delta[0] for x in coordinates.T[:, 0]])
moved_antennas_y = np.array([ y-delta[1] for y in coordinates.T[:, 1]])
moved_antennas_z = np.array([ z+delta[2] for z in lat_long[:,2]])
shifted_coordinates = np.array([moved_antennas_x, moved_antennas_y, moved_antennas_z])

In [ ]:
shifted_coordinates

In [48]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern
from sionna.constants import PI

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [114]:
# Load integrated scene
scene = load_scene("../../scenes/1&1_proto/duss_proto_1&1.xml")

In [115]:
scene.get("elm__4").radio_material = "itu_concrete"

In [116]:
epsilon_r_complex = scene.get("elm__4").radio_material.complex_relative_permittivity

In [ ]:
fc = 3.5e9
scene.frequency = fc # in Hz; implicitly updates RadioMaterials
scene.bandwidth = 7.2e6

scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)
print(f"Bandwidth: ", scene.bandwidth.numpy()/1e6, "[MHz]")

In [118]:
da = 1
M = 4
N = 4

#scene = load_scene(sionna.rt.scene.etoile)

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=N,
                             num_cols=M,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="iso",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="dipole",
                             polarization="V")
                             #polarization="cross")

In [ ]:
transmitters = [
    {
        "name":f"antenna_{i}",
        "position":shifted_coordinates.T[i],
        "orientation":angles_antennas[i],
        "power_dbm":100
    } for i in range(len(shifted_coordinates.T[:]))
]

transmitters 



In [ ]:
for i in range(len(transmitters)):
    tx = Transmitter(name=transmitters[i]["name"],
                 position=transmitters[i]["position"],
                 power_dbm=transmitters[i]["power_dbm"])

    scene.add(tx)
    print(transmitters[i]["name"])
    #scene.add(rx)

In [ ]:
cm_new = scene.coverage_map(max_depth=3,           # Maximum number of ray scene interactions
                        num_samples=int(10e6),
                        cm_center=[3.45397483e+05,5.68351812e+06,40],
                        cm_orientation=[0,0,0],
                        cm_size=[1000,1000],
                        cm_cell_size=[1,1])

In [ ]:
import matplotlib as mpl 
pos, cell_ids = cm_new.sample_positions(
          num_pos=10,
          metric="rss",
          min_dist=10,
          max_dist=200,
          tx_association=True)

fig = cm_new.show(metric="rss", tx=0);

# Visualize sampled positions
for tx, ids in enumerate(cell_ids):
    fig.axes[0].plot(ids[:,0], ids[:,1],
                     marker='x',
                     linestyle='',
                     color=mpl.colormaps['Dark2'].colors[tx])

In [ ]:
# Visualize received signal strength (RSS)
cm_new.show(metric="rss", tx=0);

# Visulaize SINR
cm_new.show(metric="sinr", tx=0);

cm_new.show_association("rss");